In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
from pandas import plotting
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
cwd = os.getcwd()
os.chdir('/content/drive/My Drive/Colab Notebooks/janataHackHealthCareII')

## Data Import

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,a,6,X,3,radiotherapy,Q,F,4.0,86499,23.0,Emergency,Moderate,3,41-50,4144.0,11-20
318434,318435,24,a,1,X,2,anesthesia,Q,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40
318435,318436,7,a,4,X,3,gynecology,R,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20
318436,318437,11,b,2,Y,3,anesthesia,Q,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20


## Exploratory Data Analysis


In [ ]:
train['Stay'].value_counts()
#this indicates a skew in the dataset

21-30                 87491
11-20                 78139
31-40                 55159
51-60                 35018
0-10                  23604
41-50                 11743
71-80                 10254
More than 100 Days     6683
81-90                  4838
91-100                 2765
61-70                  2744
Name: Stay, dtype: int64

## Data Pre-Processing


In [ ]:
train

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,a,6,X,3,radiotherapy,Q,F,4.0,86499,23.0,Emergency,Moderate,3,41-50,4144.0,11-20
318434,318435,24,a,1,X,2,anesthesia,Q,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40
318435,318436,7,a,4,X,3,gynecology,R,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20
318436,318437,11,b,2,Y,3,anesthesia,Q,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20


In [ ]:
'''## Feature Engineering Part 1 
## Grouping Based on Patient ID
train = train.sort_values(by = ['patientid']).reset_index(drop=True)
train['Group_First']= 0
train['Group_First'][0]=1
train['Group_Last']=0
train['Group']=0
sum = 0 
totalSize = (train.shape[0])
for i in tqdm(range(1,totalSize)):
  if (abs(train['patientid'][i] - train['patientid'][i-1]==1)):
    #sum+=1
    #train['Group'][i] = sum
    train['Group_First'][i] = 1
    train['Group_Last'][i-1] = 1'''

"## Feature Engineering Part 1 \n## Grouping Based on Patient ID\ntrain = train.sort_values(by = ['patientid']).reset_index(drop=True)\ntrain['Group_First']= 0\ntrain['Group_First'][0]=1\ntrain['Group_Last']=0\ntrain['Group']=0\nsum = 0 \ntotalSize = (train.shape[0])\nfor i in tqdm(range(1,totalSize)):\n  if (abs(train['patientid'][i] - train['patientid'][i-1]==1)):\n    #sum+=1\n    #train['Group'][i] = sum\n    train['Group_First'][i] = 1\n    train['Group_Last'][i-1] = 1"

In [ ]:
'''## Feature engineering Part 1
## add all money spent by these patients
train['totalMoney']= 0
le = preprocessing.LabelEncoder()
le.fit(train['patientid'])
patients = list(le.classes_)
for i in tqdm(range(len(patients))):
  conditionChecker = train['patientid']==patients[i]
  totalMoney=train.loc[conditionChecker]['Admission_Deposit'].sum()
  index = train[conditionChecker].index
  train.loc[index,'totalMoney'] = totalMoney # check this
train
#df.loc([train['patientid']])'''

"## Feature engineering Part 1\n## add all money spent by these patients\ntrain['totalMoney']= 0\nle = preprocessing.LabelEncoder()\nle.fit(train['patientid'])\npatients = list(le.classes_)\nfor i in tqdm(range(len(patients))):\n  conditionChecker = train['patientid']==patients[i]\n  totalMoney=train.loc[conditionChecker]['Admission_Deposit'].sum()\n  index = train[conditionChecker].index\n  train.loc[index,'totalMoney'] = totalMoney # check this\ntrain\n#df.loc([train['patientid']])"

In [ ]:
## feature engineering - summing
train['Bill_per_patient'] = train.groupby('patientid')['Admission_Deposit'].transform('sum')
test['Bill_per_patient']  = test.groupby('patientid')['Admission_Deposit'].transform('sum')

In [ ]:
display(train[['Severity of Illness','case_id','patientid']].head())

train['Min_Severity_of_Illness'] = train.groupby('patientid')['Severity of Illness'].transform('min')
test['Min_Severity_of_Illness']  = test.groupby('patientid')['Severity of Illness'].transform('min')
display(train.head(20))
display(test.head(20))

,Severity of Illness,case_id,patientid
0,Extreme,1,31397
1,Extreme,2,31397
2,Extreme,3,31397
3,Extreme,4,31397
4,Extreme,5,31397


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,Bill_per_patient,Min_Severity_of_Illness
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10,83314.0,Extreme
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50,83314.0,Extreme
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40,83314.0,Extreme
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50,83314.0,Extreme
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50,83314.0,Extreme
5,6,23,a,6,X,2,anesthesia,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,4449.0,11-20,83314.0,Extreme
6,7,32,f,9,Y,1,radiotherapy,S,B,3.0,31397,7.0,Emergency,Extreme,2,51-60,6167.0,0-10,83314.0,Extreme
7,8,23,a,6,X,4,radiotherapy,Q,F,3.0,31397,7.0,Trauma,Extreme,2,51-60,5571.0,41-50,83314.0,Extreme
8,9,1,d,10,Y,2,gynecology,R,B,4.0,31397,7.0,Trauma,Extreme,2,51-60,7223.0,51-60,83314.0,Extreme
9,10,10,e,1,X,2,gynecology,S,E,3.0,31397,7.0,Trauma,Extreme,2,51-60,6056.0,31-40,83314.0,Extreme


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Bill_per_patient,Min_Severity_of_Illness
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0,28765.0,Moderate
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0,28765.0,Moderate
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0,28765.0,Moderate
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0,28765.0,Moderate
4,318443,28,b,11,X,2,gynecology,R,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0,28765.0,Moderate
5,318444,23,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,2,71-80,4659.0,28765.0,Moderate
6,318445,26,b,2,Y,2,gynecology,Q,D,2.0,17006,2.0,Trauma,Moderate,2,71-80,4167.0,28765.0,Moderate
7,318446,25,e,1,X,4,gynecology,S,E,3.0,95946,NaN,Emergency,Moderate,2,31-40,4396.0,28755.0,Moderate
8,318447,23,a,6,X,4,gynecology,Q,F,3.0,95946,NaN,Trauma,Moderate,2,31-40,4088.0,28755.0,Moderate
9,318448,23,a,6,X,3,gynecology,Q,F,4.0,95946,NaN,Urgent,Moderate,2,31-40,3925.0,28755.0,Moderate


In [ ]:
## Try dropping ID and PATIENTID
train = train.drop(columns = ['case_id','patientid'])
test = test.drop(columns = ['case_id','patientid'])

In [ ]:
train.columns

Index(['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'City_Code_Patient', 'Type of Admission', 'Severity of Illness',
       'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay',
       'Bill_per_patient', 'Min_Severity_of_Illness'],
      dtype='object')

In [ ]:
le = preprocessing.LabelEncoder()
forEncoding = ['Age','Stay','Min_Severity_of_Illness']
for i in forEncoding:
  le.fit(train[i])
  x = le.transform(train[i])
  train[i] = x

In [ ]:
le = preprocessing.LabelEncoder()
forEncoding = ['Age','Min_Severity_of_Illness']
for i in forEncoding:
  le.fit(test[i])
  x = le.transform(test[i])
  test[i] = x

In [ ]:
train

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,Bill_per_patient,Min_Severity_of_Illness
0,8,c,3,Z,3,radiotherapy,R,F,2.0,7.0,Emergency,Extreme,2,5,4911.0,0,83314.0,0
1,2,c,5,Z,2,radiotherapy,S,F,2.0,7.0,Trauma,Extreme,2,5,5954.0,4,83314.0,0
2,10,e,1,X,2,anesthesia,S,E,2.0,7.0,Trauma,Extreme,2,5,4745.0,3,83314.0,0
3,26,b,2,Y,2,radiotherapy,R,D,2.0,7.0,Trauma,Extreme,2,5,7272.0,4,83314.0,0
4,26,b,2,Y,2,radiotherapy,S,D,2.0,7.0,Trauma,Extreme,2,5,5558.0,4,83314.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,6,a,6,X,3,radiotherapy,Q,F,4.0,23.0,Emergency,Moderate,3,4,4144.0,1,4144.0,2
318434,24,a,1,X,2,anesthesia,Q,E,4.0,8.0,Urgent,Moderate,4,8,6699.0,3,6699.0,2
318435,7,a,4,X,3,gynecology,R,F,4.0,10.0,Emergency,Minor,3,7,4235.0,1,4235.0,1
318436,11,b,2,Y,3,anesthesia,Q,D,3.0,8.0,Trauma,Minor,5,1,3761.0,1,3761.0,1


In [ ]:
##Manual Label Encoding 
##
#train['Severity of Illness'].value_counts()
#train['Type of Admission'].value_counts()
#cleanup_nums = {"Type of Admission":     {"Trauma": 0, "Emergency": 2, 
#                                          "Urgent": 1},
#                "Severity of Illness":   {"Moderate":0,"Minor":1,"Extreme":2}}
#train.replace(cleanup_nums, inplace=True)
#test.replace(cleanup_nums,inplace=True)

In [ ]:
train

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,Bill_per_patient,Min_Severity_of_Illness
0,8,c,3,Z,3,radiotherapy,R,F,2.0,7.0,Emergency,Extreme,2,5,4911.0,0,83314.0,0
1,2,c,5,Z,2,radiotherapy,S,F,2.0,7.0,Trauma,Extreme,2,5,5954.0,4,83314.0,0
2,10,e,1,X,2,anesthesia,S,E,2.0,7.0,Trauma,Extreme,2,5,4745.0,3,83314.0,0
3,26,b,2,Y,2,radiotherapy,R,D,2.0,7.0,Trauma,Extreme,2,5,7272.0,4,83314.0,0
4,26,b,2,Y,2,radiotherapy,S,D,2.0,7.0,Trauma,Extreme,2,5,5558.0,4,83314.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,6,a,6,X,3,radiotherapy,Q,F,4.0,23.0,Emergency,Moderate,3,4,4144.0,1,4144.0,2
318434,24,a,1,X,2,anesthesia,Q,E,4.0,8.0,Urgent,Moderate,4,8,6699.0,3,6699.0,2
318435,7,a,4,X,3,gynecology,R,F,4.0,10.0,Emergency,Minor,3,7,4235.0,1,4235.0,1
318436,11,b,2,Y,3,anesthesia,Q,D,3.0,8.0,Trauma,Minor,5,1,3761.0,1,3761.0,1


In [ ]:
trainStem = pd.DataFrame()
testStem = pd.DataFrame()
trainStem['Age'] = train['Age']
trainStem['Stay'] = train['Stay']
trainStem['Min_Severity_of_Illness']= train['Min_Severity_of_Illness']
trainStem['Bill_per_patient']= train['Bill_per_patient']


testStem['Age'] = test['Age']
testStem['Min_Severity_of_Illness']= test['Min_Severity_of_Illness']
testStem['Bill_per_patient']= test['Bill_per_patient']


In [ ]:
## FEATURE engineering


In [ ]:
forEncoding = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'City_Code_Patient','Severity of Illness','Type of Admission',
       'Visitors with Patient', 'Admission_Deposit']
train = pd.get_dummies(train[forEncoding])

In [ ]:
forEncoding = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'City_Code_Patient','Severity of Illness','Type of Admission',
       'Visitors with Patient', 'Admission_Deposit']
test = pd.get_dummies(test[forEncoding])

In [ ]:
#train.to_csv('encodedTrain.csv',index=False)
train

,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,City_Code_Patient,Visitors with Patient,Admission_Deposit,Hospital_type_code_a,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,Hospital_region_code_X,Hospital_region_code_Y,Hospital_region_code_Z,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Ward_Type_P,Ward_Type_Q,Ward_Type_R,Ward_Type_S,Ward_Type_T,Ward_Type_U,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,Severity of Illness_Extreme,Severity of Illness_Minor,Severity of Illness_Moderate,Type of Admission_Emergency,Type of Admission_Trauma,Type of Admission_Urgent
0,8,3,3,2.0,7.0,2,4911.0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0
1,2,5,2,2.0,7.0,2,5954.0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0
2,10,1,2,2.0,7.0,2,4745.0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,26,2,2,2.0,7.0,2,7272.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0
4,26,2,2,2.0,7.0,2,5558.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,6,6,3,4.0,23.0,3,4144.0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
318434,24,1,2,4.0,8.0,4,6699.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
318435,7,4,3,4.0,10.0,3,4235.0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
318436,11,2,3,3.0,8.0,5,3761.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0


In [ ]:
train = pd.concat([train,trainStem],axis=1)
test= pd.concat([test,testStem],axis=1)

In [ ]:
train.isnull().sum()

Hospital_code                           0
City_Code_Hospital                      0
Available Extra Rooms in Hospital       0
Bed Grade                             113
City_Code_Patient                    4532
Visitors with Patient                   0
Admission_Deposit                       0
Hospital_type_code_a                    0
Hospital_type_code_b                    0
Hospital_type_code_c                    0
Hospital_type_code_d                    0
Hospital_type_code_e                    0
Hospital_type_code_f                    0
Hospital_type_code_g                    0
Hospital_region_code_X                  0
Hospital_region_code_Y                  0
Hospital_region_code_Z                  0
Department_TB & Chest disease           0
Department_anesthesia                   0
Department_gynecology                   0
Department_radiotherapy                 0
Department_surgery                      0
Ward_Type_P                             0
Ward_Type_Q                       

In [ ]:
## imputing missing values with -1
train.fillna(-1.0, inplace=True)
test.fillna(-1.0,inplace=True)

In [ ]:
Y = train['Stay']
X = train.drop(columns = ['Stay'])

In [ ]:
sm = SMOTE(sampling_strategy='minority', random_state=7)

# Fit the model to generate the data.
X, Y = sm.fit_sample(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
type(X_train)

numpy.ndarray

## Initial Model Prototyping

In [ ]:
lg=LGBMClassifier(n_estimators=500)
lg.fit(X_train,Y_train)
Y_predict=lg.predict(X_test)

print(f1_score(Y_test,Y_predict, average='weighted'))
print(accuracy_score(Y_test,Y_predict))

0.5253159231177925
0.5480858662896685


In [ ]:
import pickle
import joblib
filename = 'initial.sav'
joblib.dump(lg, filename)

['initial.sav']

In [ ]:
loaded_model = joblib.load('initial.sav')

In [ ]:
Y_test_predict=loaded_model.predict(test)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
le.fit(train['Stay'])

LabelEncoder()

In [ ]:
Y_test_predict=le.inverse_transform(Y_test_predict)

In [ ]:
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission['Stay']=Y_test_predict

In [ ]:
sample_submission

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,21-30
137055,455494,11-20


In [ ]:
sample_submission.to_csv('vetriBaselineLabelEncode.csv', index=False)

## Stratified K-Fold

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
## feature engineering - summing
train['Bill_per_patient'] = train.groupby('patientid')['Admission_Deposit'].transform('sum')
test['Bill_per_patient']  = test.groupby('patientid')['Admission_Deposit'].transform('sum')

In [ ]:
display(train[['Severity of Illness','case_id','patientid']].head())

train['Min_Severity_of_Illness'] = train.groupby('patientid')['Severity of Illness'].transform('min')
train['Min_Admission'] = train.groupby('patientid')['Type of Admission'].transform('min')
train['MaxDepartment'] = train.groupby('patientid')['Department'].transform('max')
train['MinRelatives'] = train.groupby('patientid')['Visitors with Patient'].transform('min')
#train['MaxRealtives'] = train.groupby('patientid')['Visitors with Patient'].transform('max')

test['MaxDepartment'] = test.groupby('patientid')['Department'].transform('max')
test['Min_Admission'] = test.groupby('patientid')['Type of Admission'].transform('min')
test['Min_Severity_of_Illness']  = test.groupby('patientid')['Severity of Illness'].transform('min')
test['MinRelatives'] = test.groupby('patientid')['Visitors with Patient'].transform('min')
display(train.head(20))
display(test.head(20))

,Severity of Illness,case_id,patientid
0,Extreme,1,31397
1,Extreme,2,31397
2,Extreme,3,31397
3,Extreme,4,31397
4,Extreme,5,31397


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,Bill_per_patient,Min_Severity_of_Illness,Min_Admission,MaxDepartment,MinRelatives
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10,83314.0,Extreme,Emergency,radiotherapy,2
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50,83314.0,Extreme,Emergency,radiotherapy,2
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40,83314.0,Extreme,Emergency,radiotherapy,2
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50,83314.0,Extreme,Emergency,radiotherapy,2
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50,83314.0,Extreme,Emergency,radiotherapy,2
5,6,23,a,6,X,2,anesthesia,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,4449.0,11-20,83314.0,Extreme,Emergency,radiotherapy,2
6,7,32,f,9,Y,1,radiotherapy,S,B,3.0,31397,7.0,Emergency,Extreme,2,51-60,6167.0,0-10,83314.0,Extreme,Emergency,radiotherapy,2
7,8,23,a,6,X,4,radiotherapy,Q,F,3.0,31397,7.0,Trauma,Extreme,2,51-60,5571.0,41-50,83314.0,Extreme,Emergency,radiotherapy,2
8,9,1,d,10,Y,2,gynecology,R,B,4.0,31397,7.0,Trauma,Extreme,2,51-60,7223.0,51-60,83314.0,Extreme,Emergency,radiotherapy,2
9,10,10,e,1,X,2,gynecology,S,E,3.0,31397,7.0,Trauma,Extreme,2,51-60,6056.0,31-40,83314.0,Extreme,Emergency,radiotherapy,2


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Bill_per_patient,MaxDepartment,Min_Admission,Min_Severity_of_Illness,MinRelatives
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0,28765.0,gynecology,Emergency,Moderate,2
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0,28765.0,gynecology,Emergency,Moderate,2
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0,28765.0,gynecology,Emergency,Moderate,2
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0,28765.0,gynecology,Emergency,Moderate,2
4,318443,28,b,11,X,2,gynecology,R,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0,28765.0,gynecology,Emergency,Moderate,2
5,318444,23,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,2,71-80,4659.0,28765.0,gynecology,Emergency,Moderate,2
6,318445,26,b,2,Y,2,gynecology,Q,D,2.0,17006,2.0,Trauma,Moderate,2,71-80,4167.0,28765.0,gynecology,Emergency,Moderate,2
7,318446,25,e,1,X,4,gynecology,S,E,3.0,95946,NaN,Emergency,Moderate,2,31-40,4396.0,28755.0,gynecology,Emergency,Moderate,2
8,318447,23,a,6,X,4,gynecology,Q,F,3.0,95946,NaN,Trauma,Moderate,2,31-40,4088.0,28755.0,gynecology,Emergency,Moderate,2
9,318448,23,a,6,X,3,gynecology,Q,F,4.0,95946,NaN,Urgent,Moderate,2,31-40,3925.0,28755.0,gynecology,Emergency,Moderate,2


In [ ]:
## Try dropping ID and PATIENTID
train = train.drop(columns = ['case_id','patientid'])
test = test.drop(columns = ['case_id','patientid'])

In [ ]:
'''le = preprocessing.LabelEncoder()
forEncoding = ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Age','Severity of Illness','Stay']
for i in forEncoding:
  le.fit(train[i])
  x = le.transform(train[i])
  train[i] = x'''

"le = preprocessing.LabelEncoder()\nforEncoding = ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Age','Severity of Illness','Stay']\nfor i in forEncoding:\n  le.fit(train[i])\n  x = le.transform(train[i])\n  train[i] = x"

In [ ]:
'''le = preprocessing.LabelEncoder()
forEncoding = ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Age','Severity of Illness']
for i in forEncoding:
  le.fit(test[i])
  x = le.transform(test[i])
  test[i] = x'''

"le = preprocessing.LabelEncoder()\nforEncoding = ['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Age','Severity of Illness']\nfor i in forEncoding:\n  le.fit(test[i])\n  x = le.transform(test[i])\n  test[i] = x"

In [ ]:
le = preprocessing.LabelEncoder()
forEncoding = ['Age','Stay','Min_Severity_of_Illness']
for i in forEncoding:
  le.fit(train[i])
  x = le.transform(train[i])
  train[i] = x

In [ ]:
le = preprocessing.LabelEncoder()
forEncoding = ['Age','Min_Severity_of_Illness']
for i in forEncoding:
  le.fit(test[i])
  x = le.transform(test[i])
  test[i] = x

In [ ]:
trainStem = pd.DataFrame()
testStem = pd.DataFrame()
trainStem['Age'] = train['Age']
trainStem['Stay'] = train['Stay']
trainStem['successCoeff'] = train['Admission_Deposit']/(train['Age']+1)
#trainStem['Min_Severity_of_Illness']= train['Min_Severity_of_Illness']
#trainStem['Bill_per_patient']= train['Bill_per_patient']
#trainStem['Min_Admission']= train['Min_Admission']

#testStem['Min_Admission']= test['Min_Admission']
testStem['Age'] = test['Age']
testStem['successCoeff'] = test['Admission_Deposit']/(test['Age']+1)
#testStem['Min_Severity_of_Illness']= test['Min_Severity_of_Illness']
#testStem['Bill_per_patient']= test['Bill_per_patient']

In [ ]:
forEncoding = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital','MaxDepartment',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade','MinRelatives',
       'City_Code_Patient','Severity of Illness','Type of Admission','Bill_per_patient','Min_Admission',
       'Visitors with Patient', 'Admission_Deposit']
train = pd.get_dummies(train[forEncoding])

''''Type of Admission', 'Severity of Illness','''

"'Type of Admission', 'Severity of Illness',"

In [ ]:
forEncoding = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital','MaxDepartment',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade','MinRelatives',
       'City_Code_Patient','Severity of Illness','Type of Admission','Bill_per_patient','Min_Admission',
       'Visitors with Patient', 'Admission_Deposit']
test = pd.get_dummies(test[forEncoding])

In [ ]:
train = pd.concat([train,trainStem],axis=1)
test= pd.concat([test,testStem],axis=1)

In [ ]:
## imputing missing values with -1
train.fillna(method='ffill',inplace=True)
test.fillna(method='ffill',inplace=True)
#train.fillna(-1.0, inplace=True)
#test.fillna(-1.0,inplace=True)

In [ ]:
Y = train['Stay']
X = train.drop(columns = ['Stay'])

In [ ]:
sm = SMOTE(sampling_strategy='minority', random_state=7)

# Fit the model to generate the data.
X, Y = sm.fit_sample(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

In [ ]:
lg=LGBMClassifier(n_estimators=500)

In [ ]:
#lg = LGBMClassifier(boosting_type='gbdt', learning_rate=0.1, n_estimators=500, 
#                      min_child_samples=20, random_state=1994,  n_jobs=-1, silent=False)

In [ ]:
kf=KFold(n_splits=10, random_state=100, shuffle=True)

y_test_predict=0
f1_score1 = 0
acc_score1 = 0
j=1
result={}
for i, (train_index, test_index) in enumerate(kf.split(train)):
    
   Y_train, Y_valid = Y.iloc[train_index], Y.iloc[test_index]
   X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
   
   print( "\nFold ", j)
    
   model=lg.fit(X_train,Y_train)
   pred = model.predict(X_valid)
   
   print(f1_score(Y_valid,pred, average='weighted'))
   print(accuracy_score(Y_valid,pred)) 
   f1_score1+=f1_score(Y_valid,pred, average='weighted')
   acc_score1+=accuracy_score(Y_valid,pred)

   y_test_predict+=model.predict(test)  
   result[j]=model.predict(test)
   j+=1
   
    
results=y_test_predict/10  
results=np.round(results, decimals=0)
print(f1_score1/10)

d = pd.DataFrame()
for i in range(1, 11):
    d = pd.concat([d,pd.DataFrame(result[i])],axis=1)
d.columns=['1','2','3','4','5','6','7','8','9','10']


Fold  1


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.39393665973642444
0.42679939706067077

Fold  2
0.3988039592988137
0.43289159653309883

Fold  3
0.39973239404649674
0.4327973872629067

Fold  4
0.39716236114768433
0.431447054390152

Fold  5
0.39382919178613895
0.4265481723401583

Fold  6
0.4002928150682513
0.4337080768747645

Fold  7
0.3987753741061307
0.43166687602060044

Fold  8
0.39202438554938623
0.4255118703680442

Fold  9
0.3946060958527535
0.42690701253022645

Fold  10
0.3959095055201467
0.42998461200263793
0.39650727421122267


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
le.fit(train['Stay'])

LabelEncoder()

In [ ]:
re = d.mode(axis=1)[0]
print(re)

0         0.0
1         5.0
2         2.0
3         2.0
4         5.0
         ... 
137052    2.0
137053    0.0
137054    2.0
137055    1.0
137056    3.0
Name: 0, Length: 137057, dtype: float64


In [ ]:
re = re.astype(int)

In [ ]:
Y_test_predict=le.inverse_transform(re)

In [ ]:
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission['Stay']=Y_test_predict
sample_submission.to_csv('stratifiedSuccessCoefficientOGMODEL.csv', index=False)